<a href="https://colab.research.google.com/github/pquimerch08/tp1_collab_pol_quimerch/blob/main/tp1_pol_quimerch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TP: PySpark pour analyse de données de films.

In [164]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round, split, explode, regexp_replace, when, ceil


## 1. Prise en main: Préparation des données

#### Initialiser la session Spark

In [143]:
spark = SparkSession.builder.appName("MoviesData").getOrCreate()

#### Charger le fichier CSV dans un DataFrame

In [144]:
file_path = "Rotten Tomatoes Movies.csv"
df = spark.read.option("header", "true").csv(file_path)

In [145]:
df.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|         movie_title|          movie_info|   critics_consensus|              rating|               genre|           directors|             writers|                cast|    in_theaters_date|   on_streaming_date|  runtime_in_minutes|         studio_name|  tomatometer_status|  tomatometer_rating|   tomatometer_count|audience_rating|      audience_count|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------

#### Supprimer les lignes contenant des valeurs nulles

In [146]:
df_clean = df.dropna()

In [147]:
df_clean.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|         movie_title|          movie_info|   critics_consensus|              rating|               genre|           directors|             writers|                cast|    in_theaters_date|   on_streaming_date|  runtime_in_minutes|         studio_name|  tomatometer_status|  tomatometer_rating|   tomatometer_count|audience_rating|      audience_count|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------

In [148]:
from pyspark.sql.functions import col, to_date

#### Convertir les colonnes in_theaters_date et on_streaming_date au format yyyy-MM-dd

In [149]:
df_clean = df_clean \
    .withColumn("in_theaters_date", to_date(col("in_theaters_date"), "yyyy-MM-dd")) \
    .withColumn("on_streaming_date", to_date(col("on_streaming_date"), "yyyy-MM-dd")) \
    .na.drop(subset=["in_theaters_date", "on_streaming_date"]) #Supprimer les lignes avec des valeurs nulles ou invalides


In [150]:
df_clean.show(10)

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|           directors|             writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|Percy Jackson & t...|A teenager discov...|Tho

## 2. Intermediaire: Manipulation des données & analyse

#### Les films ayant une note très basse (tomatometer_rating < 20)

In [151]:
# Filtrer les films avec une note inférieure à 20
df_filtered = df_clean.filter(col("tomatometer_rating") < 20)

# Trier les films dans l'ordre croissant des notes
df_sorted = df_filtered.orderBy("tomatometer_rating", ascending=True)

# Afficher les résultats (les 50 premiers films)
df_sorted.show(50)

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|           directors|             writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|   Merci Docteur Rey|Andrew Litvack's ...|Thi

#### Les films sortis (en cinéma) après l’année 2000

In [152]:
# Filtrer les films sortis après l'année 2000
df_films_after_2000 = df_clean.filter(col("in_theaters_date") > "2000-01-01")

#Tri dans l'ordre croissant
df_sorted = df_films_after_2000.orderBy("in_theaters_date", ascending=True)

# Afficher les résultats
df_sorted.show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|           directors|             writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|           Supernova|Supernova chronic...|Thi

#### La note moyenne des films par studio

In [153]:
df_avg_studio = df_clean.groupBy("studio_name").agg(
    round(avg("tomatometer_rating"), 1).alias("average_tomatometer_rating")
)

#Trier notes par ordre décroissant
df_sorted = df_avg_studio.orderBy("average_tomatometer_rating", ascending=False)

# Afficher les résultats
df_sorted.show()

+--------------------+--------------------------+
|         studio_name|average_tomatometer_rating|
+--------------------+--------------------------+
|      Ealing Studios|                     100.0|
|Distribution Company|                     100.0|
|Nagoya Broadcasti...|                     100.0|
| Shore International|                     100.0|
|Zenith Internatio...|                     100.0|
|    Grasshopper Film|                     100.0|
|    Showtime Network|                     100.0|
|                 NCM|                     100.0|
|La Casa de Produc...|                     100.0|
|Majestic Films In...|                     100.0|
|   Itami Productions|                     100.0|
|         Park Circus|                     100.0|
|Playground Produc...|                     100.0|
|      BFI Production|                     100.0|
|               Xenon|                     100.0|
|    Skouras Pictures|                     100.0|
|     Edward Harrison|                     100.0|


 #### La note moyenne des films par directeur

In [154]:
df_avg_directors = df_clean.groupBy("directors").agg(
    round(avg("tomatometer_rating"), 1).alias("average_tomatometer_rating")
)

#Trier notes par ordre décroissant
df_sorted = df_avg_directors.orderBy("average_tomatometer_rating", ascending=False)

# Afficher les résultats
df_sorted.show()

+--------------------+--------------------------+
|           directors|average_tomatometer_rating|
+--------------------+--------------------------+
|        Bill Forsyth|                     100.0|
|     Kenji Mizoguchi|                     100.0|
|           Lixin Fan|                     100.0|
|       Jorge Gaggero|                     100.0|
|        Rachel Lears|                     100.0|
|     Chantal Akerman|                     100.0|
|         James Whale|                     100.0|
|        Delbert Mann|                     100.0|
|        Robert Hamer|                     100.0|
|          Juzo Itami|                     100.0|
|Shlomi Elkabetz, ...|                     100.0|
|                One9|                     100.0|
|John Huston, John...|                     100.0|
|Rolf Bickel, Diet...|                     100.0|
|    Jayro Bustamante|                     100.0|
|      Ernst Lubitsch|                     100.0|
|       Mark Sandrich|                     100.0|


## 3. Avancé: Utilisation de fonctions avancées

#### Diviser les genres multiples d’une colonne en genres individuels

In [156]:
df_with_individual_genres = df_clean \
    .withColumn("genre", explode(split(col("genre"), ",\s*")))

df_with_individual_genres.show(5)

+--------------------+--------------------+--------------------+------+--------------------+-----------------+-----------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|        directors|          writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+-----------------+-----------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|Percy Jackson & t...|A teenager discov...|Though it may see...|

#### Calculer la durée moyenne des films pour chaque genre

In [165]:
# Calcul de la durée moyenne par genre, arrondi à l'entier supérieur
average_runtime_by_genre = df_with_individual_genres \
    .groupBy("genre") \
    .agg(
        ceil(avg(col("runtime_in_minutes").cast("float"))).alias("average_runtime")
    ) \
    .orderBy("average_runtime", ascending=False)

# Afficher les résultats
average_runtime_by_genre.show(truncate=False)

+-------------------------+---------------+
|genre                    |average_runtime|
+-------------------------+---------------+
|Classics                 |116            |
|Western                  |116            |
|Faith & Spirituality     |112            |
|Action & Adventure       |110            |
|Drama                    |110            |
|Romance                  |109            |
|Art House & International|108            |
|Science Fiction & Fantasy|108            |
|Mystery & Suspense       |107            |
|Musical & Performing Arts|106            |
|Sports & Fitness         |105            |
|Cult Movies              |102            |
|Comedy                   |101            |
|Anime & Manga            |100            |
|Television               |100            |
|Gay & Lesbian            |100            |
|Special Interest         |99             |
|Horror                   |99             |
|Documentary              |97             |
|Kids & Family            |96   